<a href="https://colab.research.google.com/github/minjeon99/ESAA-10th-OB/blob/Week2/w2_fri_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___


- 확률적 경사 하강법(SGD), 미니 배치 경사 하강법 등을 사용하면 빠르게 학습할 수 있다.

(+) 정규 방정식, SVD: 특성 개수에 따라 계산 복잡도가 매우 빠르게 증가하여 사용할 수 없다.

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

- 훈련 오차는 감소하고 있고 검증 오차보다 작다면 overfitting => 규제를 적용하거나 overfitting이 되기 전에 학습을 멈춘다.

(+) 학습률이 너무 높아서 알고리즘이 발산하는 경우 - 훈련 오차도 증가하고 있다면 이 경우에 해당할 것이다.

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

- 훈련 오차와 검증 오차가 모두 높았다는 것은 모델이 underfitting이라는 것이고, 따라서 규제 α를 줄여야 한다.

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

1. 일반 선형 회귀 대신 릿지: 규제가 있는(릿지 회귀) 경우가 없는 경우보다 overfitting을 방지할 수 있으므로 일반적으로 성능이 높다.
2. 릿지 대신 라쏘: 실제로 유의미한 특성이 몇 개만 있다고 판단될 경우, 피처 선택의 효과가 있는 Lasso를 사용한다.
3. 라쏘 대신 엘라스틱넷: 특성 수가 훈련 샘플 수보다 많거나, 특성 몇 개가 강하게 연관된 경우 라쏘 회귀가 문제를 일으킬 수 있다. 따라서 일반적으로는 엘라스틱넷이 선호된다.

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 iris 데이터를 활용해 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



- 데이터 로드

In [45]:
from sklearn.datasets import load_iris

iris = load_iris()

X = iris['data']
y = iris['target']

- 편향 추가

In [46]:
X_b = np.c_[np.ones([len(X), 1]), X]

- train_test_split

In [47]:
import numpy as np

In [48]:
test_ratio = 0.2
val_ratio = 0.2

total_size = len(X_b)
test_size = int(total_size * test_ratio)
validation_size = int(total_size * val_ratio)
train_size = total_size - test_size - validation_size

rnd_idx = np.random.permutation(total_size)

X_train = X_b[rnd_idx[:train_size]]
y_train = y[rnd_idx[:train_size]]

X_valid = X_b[rnd_idx[train_size:-test_size]]
y_valid = y[rnd_idx[train_size:-test_size]]

X_test = X_b[rnd_idx[-test_size:]]
y_test = y[rnd_idx[-test_size:]]

- target vector 만들기

In [49]:
def to_one_hot(y):
    n_classes = y.max() + 1
    m = len(y)
    Y_one_hot = np.zeros((m, n_classes))
    Y_one_hot[np.arange(m), y] = 1
    return Y_one_hot

In [50]:
Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

- softmax 함수

In [51]:
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

- 입력, 출력 개수

In [52]:
n_inputs = X_train.shape[1]
n_outputs = len(np.unique(y_train))
print(n_inputs, n_outputs)

5 3


- 비용함수

In [53]:
eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    if iteration % 500 == 0:
        loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        print(iteration, loss)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error)
    Theta = Theta - eta * gradients

0 4.558126040263288
500 0.37156427144636633
1000 0.3073784225589143
1500 0.2679683001146103
2000 0.2401220460933407
2500 0.21923904342126863
3000 0.20295774209497708
3500 0.18988987527611154
4000 0.17915760784183626
4500 0.17017723522290454
5000 0.16254523933643153


In [54]:
Theta

array([[ 1.34639445,  1.52381893,  0.09115323],
       [-0.02245418, -0.55753   , -2.01652052],
       [ 1.59607572, -0.03733931, -1.39020673],
       [-2.01404411,  1.38233994,  3.25610265],
       [ 0.99601361, -0.95052932,  2.26545654]])

- 검증 세트에 대한 정확도

In [55]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

1.0

- 규제 추가

In [56]:
eta = 0.1
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1  # 규제 하이퍼파라미터

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    if iteration % 500 == 0:
        xentropy_loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
        loss = xentropy_loss + alpha * l2_loss
        print(iteration, loss)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

0 6.993777310309954
500 0.654628083930609
1000 0.6219056334277727
1500 0.5946073236621718
2000 0.5720701999218857
2500 0.5536619061569048
3000 0.5387479271747189
3500 0.5267325682149304
4000 0.5170854864842168
4500 0.5093526654945171
5000 0.5031562212946477


In [57]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9

- 조기 종료

In [59]:
eta = 0.1
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1  # 규제 하이퍼파라미터
best_loss = np.infty

Theta = np.random.randn(n_inputs, n_outputs)

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

    logits = X_valid.dot(Theta)
    Y_proba = softmax(logits)
    xentropy_loss = -np.mean(np.sum(Y_valid_one_hot * np.log(Y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = xentropy_loss + alpha * l2_loss
    if iteration % 500 == 0:
        print(iteration, loss)
    if loss < best_loss:
        best_loss = loss
    else:
        print(iteration - 1, best_loss)
        print(iteration, loss, "조기 종료!")
        break

0 2.5756165664501056
6 0.863203526475495
7 0.8686823628388703 조기 종료!


In [60]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

1.0

- 테스트 세트 정확도

In [62]:
logits = X_test.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_test)
accuracy_score

0.9333333333333333